In [1]:
from module_generator import ModelDefinitions, create_model_def
import io_utils

In [9]:
model_def=create_model_def("model_planet.yaml")
db=model_def.database_root
print(db['url'])
print(str(db['connect_args']))
print(model_def.doc_root["name"])
for m in model_def.models:
    print(m)

base_model Base model
	abstract True
people People
planet Planet
cockroachdb://maxroach@localhost:26257/bank
{'sslmode': 'disable'}
planet
people
planet


In [9]:
database_def='''from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import scoped_session, sessionmaker

## Create database engine: cockroach
db_name = '{db[name]}'
connect_args = {db[connect_args]}

engine = create_engine(
    '{db[url]}',
    connect_args=connect_args,
    echo=True                   # Log SQL queries to stdout
)

# Declarative base model to create database tables and classes
Base = declarative_base()
Base.metadata.bind = engine  # Bind engine to metadata of the base class

# Create database session object
db_session = scoped_session(sessionmaker(bind=engine, expire_on_commit=False))
Base.query = db_session.query_property()  # Used by graphql to execute queries

'''

print(database_def.format(db=model_def.database_root))


from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import scoped_session, sessionmaker

## Create database engine: cockroach
db_name = 'local cockroachdb'
connect_args = {'sslmode': 'disable'}

engine = create_engine(
    'cockroachdb://maxroach@localhost:26257/bank',
    connect_args=connect_args,
    echo=True                   # Log SQL queries to stdout
)

# Declarative base model to create database tables and classes
Base = declarative_base()
Base.metadata.bind = engine  # Bind engine to metadata of the base class

# Create database session object
db_session = scoped_session(sessionmaker(bind=engine, expire_on_commit=False))
Base.query = db_session.query_property()  # Used by graphql to execute queries




In [3]:
from shutil import copyfile
# copyfile(src, dst)
prefix="./output/"
module_name="planet"
module_path=prefix+module_name
io_utils.create_dir(module_path)
db_path=module_path+"/database"
seed_path=module_path+"/database/data"
io_utils.create_dir(seed_path)

copyfile("./templates/api.py", module_path+"/api.py")
copyfile("./templates/utils.py", module_path+"/utils.py")

'./output/planet/utils.py'

In [8]:
import os
for path in io_utils.list_files("./seeds/"+module_name):
    copyfile(path, seed_path+"/"+os.path.basename(path))

In [6]:
## process model extends
from module_generator import ModelDefinitions, create_model_def
model_def=create_model_def("model_planet.yaml")
model=model_def.models['people']

field_def='''{fld_name} = Column('{fld_name}', {fld_type}, {pk_part} doc="{field[doc]}")'''
field_builder=[]

if 'extends' in model:
    base_model=model_def.base_models[model['extends']]
    for k, fld in base_model['fields'].items():
        # print(k)
        field_builder.append(field_def.format(fld_name=k, 
                           fld_type=fld['type'].capitalize(),
                           pk_part="",
                           field=fld))
print("\n".join(field_builder))        

created = Column('created', String,  doc="Record created date.")
edited = Column('edited', String,  doc="Record last updated date.")
